## Dataset
This section loads the dataset and creates column sets and media definitions.

In [28]:
from mediamix.datasets import load_he_mmm_dataset, he_mmm_definitions, make_he_mmm_column_sets
df = load_he_mmm_dataset()
col_sets = make_he_mmm_column_sets(df)
df['is_holiday'] = df[col_sets['holiday']].sum(axis=1)

In [32]:
df.to_csv('../data/he_mmm_data.csv')

## Adstock Estimation
This section generates geometric adstock transformations - should randomize the parameters or fit using Bayesian methods.

In [2]:
import pymc
import numpy as np
from mediamix.transforms import geometric_adstock, logistic_saturation, hill_saturation

In [3]:
for col in col_sets['media_spend']:
    df[f'{col}_adstock'] = geometric_adstock(df[col].values, alpha=1, L=12, normalize=False).eval()

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

media = 'vidtr'
spend_col = f'mdsp_{media}'
adstock_col = f'mdsp_{media}_adstock'

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df.index, y=df[spend_col], name="Original Spend", line=dict(color='blue')),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=df.index, y=df[adstock_col], name="Adstocked Spend", line=dict(color='red')),
    secondary_y=True
)

fig.update_layout(
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    ),
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.update_yaxes(title_text="Media Spend ($)", secondary_y=False)
fig.update_yaxes(title_text="Adstocked Spend ($)", secondary_y=True)
fig.show()


## Saturation Estimation
This section generates hill saturation transformations - should randomize the parameters or fit using Bayesian methods.

In [5]:
# Store the saturation parameters for optimization later
saturation_params = {}

# Apply saturation transformation to adstocked media variables
for col in col_sets['media_spend']:
    adstock_col = f'{col}_adstock'
    # Scale the data to find a good k value (half saturation point)
    k_value = df[adstock_col].max() * 0.5
    s_value = 0.7  # Default slope parameter
    
    # Store parameters for later optimization
    saturation_params[col] = {'k': k_value, 's': s_value}
    
    # Apply hill transformation
    df[f'{col}_transformed'] = hill_saturation(df[adstock_col].values, k=k_value, s=s_value).eval()

In [6]:
media = 'vidtr'
spend_col = f'mdsp_{media}'
adstock_col = f'mdsp_{media}_adstock'
hill_col = f'mdsp_{media}_transformed'

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df[adstock_col],
    y=df[hill_col],
    mode='markers',
    name='Transformed vs Adstocked'
))

fig.update_layout(
    xaxis_title=f'Adstocked Spend',
    yaxis_title=f'Effectivenss',
    showlegend=False,
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()


## Combined Model
Once we have transformed columns, we can use them in a global model to attribute sales to each media channel. We will use this model for optimization as well.

We actually need two models to get the differential uplift from media spend - a base or control model without any media variables, and a media marketing model. We will then decompose the effects of the marketing spend. But we can simplify by controlling for our base variables within multivariate regression.

### Base Model
For the base model, we will use the macro-economic variables, store count, markdown discount, and whether it is a holidary or not. Five features with sales as our target.

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error

In [8]:
control_cols = col_sets['macro_economics'] + col_sets['store_count'] + col_sets['markdown_discount'] + ['is_holiday']
transformed_media_cols = [f'{col}_transformed' for col in col_sets['media_spend']]
model_features = transformed_media_cols + control_cols

X = df[model_features]
y = df['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin

class PositiveMMM(BaseEstimator, RegressorMixin):
    """Constrained regression for positive channel contributions"""
    def __init__(self, media_idx, max_iter=1000):
        self.media_idx = media_idx  # Indices of media features
        self.max_iter = max_iter
        
    def fit(self, X, y):
        X = np.column_stack([np.ones(len(X)), X])  # Add intercept
        initial_coef = np.linalg.lstsq(X, y, rcond=None)[0]
        
        # Set bounds: intercept (no bounds) + media (positive) + other features (no bounds)
        bounds = [(None, None)] + [
            (0, None) if i in self.media_idx else (None, None) 
            for i in range(X.shape[1]-1)
        ]
        
        def loss(coef):
            return np.sum((y - X.dot(coef)) ** 2)
            
        result = minimize(loss, initial_coef, 
                         bounds=bounds,
                         method='L-BFGS-B',
                         options={'maxiter': self.max_iter})
        
        self.coef_ = result.x[1:]
        self.intercept_ = result.x[0]
        return self
        
    def predict(self, X):
        return self.intercept_ + X.dot(self.coef_)

# Identify media feature indices (first n features)
transformed_media_cols = [f'{col}_transformed' for col in col_sets['media_spend']]
media_idx = [model_features.index(col) for col in transformed_media_cols]

# Initialize and fit model
model = PositiveMMM(media_idx=media_idx)
model.fit(X_train.values, y_train.values)

# Evaluate model
y_pred = model.predict(X_test.values)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Positive Constrained Model R²: {r2:.3f}")
print(f"MAE: {mae:,.0f}")

# Verify positive coefficients
coef_df = pd.DataFrame({
    'Feature': model_features,
    'Coefficient': model.coef_
}).sort_values('Coefficient', ascending=False)

print("\nPositive Coefficients Only:")
print(coef_df.query('Coefficient >= 0'))

Positive Constrained Model R²: -1.386
MAE: 71,137,343

Positive Coefficients Only:
                    Feature   Coefficient
14                mrkdn_pdm  1.401692e+09
8       mdsp_on_transformed  5.105790e+08
9      mdsp_sem_transformed  4.560394e+08
13         mrkdn_valadd_edw  1.290152e+08
11               me_gas_dpg  5.592724e+07
2      mdsp_nsp_transformed  5.095061e+07
7       mdsp_so_transformed  4.954307e+07
1     mdsp_inst_transformed  2.704922e+07
4    mdsp_audtr_transformed  1.992770e+07
15               is_holiday  1.245790e+07
6   mdsp_viddig_transformed  2.782175e+06
12                    st_ct  3.814558e+05
0       mdsp_dm_transformed  0.000000e+00
3   mdsp_auddig_transformed  0.000000e+00
5    mdsp_vidtr_transformed  0.000000e+00


### Response Curves
These should be nonlinear, but we can approximate them with a linear model.

In [10]:
# Function to calculate response curves
def calculate_response_curve(model, X_mean, col_name, points=20):
    """Generate response curve for a specific feature"""
    feature_min = max(0, X[col_name].min())
    feature_max = X[col_name].max() * 1.5  # Extend a bit beyond observed data
    
    x_points = np.linspace(feature_min, feature_max, points)
    y_responses = []
    
    for x in x_points:
        X_sim = X_mean.copy()
        X_sim[col_name] = x
        y_responses.append(model.predict(X_sim)[0])
    
    return x_points, np.array(y_responses)

def generate_response_curves(model, X_train, transformed_media_cols, model_features):
    # Get average value for all features
    X_mean = X_train.mean().to_frame().T
    # Calculate response curves for all media channels
    response_data = []
    for i, col_name in enumerate(transformed_media_cols):
        original_col = col_name.replace('_transformed', '')
        feature_index = model_features.index(col_name)
        
        x_points, y_responses = calculate_response_curve(model, X_mean, col_name)
        
        # Calculate incremental contribution
        baseline = (model.predict(X_mean)[0] - model.coef_[feature_index] * X_mean[col_name])[0]
        y_incremental = [y - baseline for y in y_responses]
        
        # Create rows for each point
        for x, y in zip(x_points, y_incremental):
            response_data.append({
                'Channel': original_col,
                'Media_Spend': x,
                'Incremental_Sales': y
            })

    return pd.DataFrame(response_data)


In [11]:
response_curves = generate_response_curves(model, X_train, transformed_media_cols, model_features)

In [12]:
import plotly.express as px

# Create DataFrame and plot with plotly express
fig = px.line(
    response_curves, 
    x='Media_Spend',
    y='Incremental_Sales',
    color='Channel',
    labels={
        'Media_Spend': 'Media Spend (Transformed)',
        'Incremental_Sales': 'Incremental Sales'
    }
)

fig.update_layout(
    width=800,
    height=400,
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()

# Calculate channel contribution
contributions = {}
for i, col in enumerate(transformed_media_cols):
    original_col = col.replace('_transformed', '')
    contribution = model.coef_[i] * X_train[col].mean()
    contributions[original_col] = contribution

# Plot contributions
contribution_df = pd.DataFrame({'Channel': list(contributions.keys()),
                              'Contribution': list(contributions.values())})
contribution_df = contribution_df.sort_values('Contribution', ascending=False)

fig = go.Figure(go.Bar(
    x=contribution_df['Contribution'],
    y=contribution_df['Channel'],
    orientation='h'
))

fig.update_layout(
    xaxis_title='Contribution',
    yaxis_title='Channel',
    width=800,
    height=400,
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()


In [13]:
## Optimization

In [14]:
from scipy import optimize
def original_to_transformed(spend_values, media_cols):
    """Convert original spend to transformed values for model prediction"""
    transformed_values = []
    
    for i, col in enumerate(media_cols):
        # Apply adstock
        adstocked = geometric_adstock(np.array([spend_values[i]])).eval()
        
        # Apply hill transformation with stored parameters
        k = saturation_params[col]['k']
        s = saturation_params[col]['s']
        transformed = hill_saturation(adstocked, k=k, s=s).eval()
        
        transformed_values.append(transformed[0])
    
    return transformed_values

def objective_function(spend_values, media_cols, total_budget, model, X_avg, feature_cols):
    """
    Objective function to maximize sales given spend allocation
    
    Args:
        spend_values: Proposed media spend values
        media_cols: Media channel columns
        total_budget: Budget constraint
        model: Trained model
        X_avg: Average feature values
        feature_indices: Indices of media features in X
        
    Returns:
        Negative predicted sales (for minimization)
    """
    # Check if spend exceeds budget
    if sum(spend_values) > total_budget:
        return 1e10  # Large penalty for exceeding budget
    
    # Convert original spend to transformed values
    transformed_values = original_to_transformed(spend_values, media_cols)
    
    # Create prediction array
    X_pred = X_avg.copy()
    for i, feature_col in enumerate(feature_cols):
        X_pred[feature_col] = transformed_values[i]
    
    # Predict sales and return negative (for minimization)
    return -model.predict(X_pred)[0]

def optimize_budget(model, media_cols, transformed_cols, total_budget, X_avg):
    """
    Optimize budget allocation across media channels
    
    Args:
        model: Trained model
        media_cols: Media channel columns
        transformed_cols: Transformed media columns
        total_budget: Total budget to allocate
        X_avg: Average feature values
        
    Returns:
        Optimized budget allocation
    """
    # Get current spend levels for initial guess
    current_spend = [df[col].mean() for col in media_cols]

    total_budget = total_budget * 0.99

    # Define bounds (min and max spend for each channel)
    bounds = [(0, total_budget) for _ in media_cols]
    
    # Additional constraint: total spend equals budget
    def budget_constraint(x):
        return total_budget - sum(x)
    
    constraints = [{'type': 'eq', 'fun': budget_constraint}]
    
    # Run optimization
    result = optimize.minimize(
        objective_function,
        current_spend,
        args=(media_cols, total_budget, model, X_avg, transformed_cols),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    return result.x

In [15]:
# Average feature values for prediction
X_avg = X.mean().to_frame().T

# Specify total budget (e.g., sum of current average spend)
current_total_spend = sum(df[col_sets['media_spend']].mean())
total_budget = current_total_spend * 1.5

In [16]:
 # Run optimization
optimal_allocation = optimize_budget(
    model, 
    col_sets['media_spend'], 
    transformed_media_cols, 
    total_budget, 
    X_avg
)

ValueError: all input array dimensions other than the specified `axis` (0) must match exactly, or be unknown (None), but along dimension 1, the inputs shapes are incompatible: [ 1  1  2  3  4  5  6  7  8  9 10 11]

In [26]:


# Compare current vs. optimized allocation
comparison = pd.DataFrame({
    'Channel': col_sets['media_spend'],
    'Current Spend': df[col_sets['media_spend']].mean().values,
    'Optimized Spend': optimal_allocation,
    'Change %': ((optimal_allocation / df[col_sets['media_spend']].mean().values) - 1) * 100
})

print("Budget Optimization Results:")
print(comparison)

# # Visualize the comparison
# comparison_melted = pd.melt(comparison,
#                            id_vars='Channel',
#                            value_vars=['Current Spend', 'Optimized Spend'], 
#                            var_name='Allocation',
#                            value_name='Spend')

# fig = px.bar(comparison_melted,
#              x='Channel',
#              y='Spend',
#              color='Allocation',
#              barmode='group',
#     )

# fig.update_layout(
#     xaxis_tickangle=45,
#     showlegend=True,
#     height=400,
#     width=800,
#     margin=dict(t=50, l=50, r=50, b=100)
# )

# fig.show()


ValueError: all input array dimensions other than the specified `axis` (0) must match exactly, or be unknown (None), but along dimension 1, the inputs shapes are incompatible: [ 1  1  2  3  4  5  6  7  8  9 10 11]